In [1]:
source("../Data Generator.r")
source("../Longtree.r")

Warning message:
"package 'glmertree' was built under R version 3.6.1"Loading required package: lme4
Warning message:
"package 'lme4' was built under R version 3.6.1"Loading required package: Matrix
Loading required package: partykit
Warning message:
"package 'partykit' was built under R version 3.6.1"Loading required package: grid
Loading required package: libcoin
Warning message:
"package 'libcoin' was built under R version 3.6.1"Loading required package: mvtnorm
Warning message:
"package 'WGCNA' was built under R version 3.6.1"Loading required package: dynamicTreeCut
Loading required package: fastcluster

Attaching package: 'fastcluster'

The following object is masked from 'package:stats':

    hclust


Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: 'WGCNA'

The following object is masked from 'package:stats':

    cor

Warning message:
"package 'pre' was built under R v

Note: the following is where a1=1,a2=--1

# Tuning Longtree

In [2]:
### training, validation and test set ###
set.seed(100)
n = 100
p = 400
imp_mod = c(1,4)
var_noise = 1
data = sim_time(n=n,p=p,imp_mod=imp_mod, var_noise=var_noise,a1=1,a2=-1)
data$time2 = (data$time)^2

# test set (used for testing performance using optimal parameters)
set.seed(101)
n_test = 100
data_test = sim_time(n=n_test,p=p,imp_mod=imp_mod, var_noise=var_noise,a1=1,a2=-1)
data_test$time2 = (data_test$time)^2

# validation set (used for tuning parameters)
set.seed(102)
n_valid = 100
data_valid = sim_time(n=n_valid,p=p,imp_mod=imp_mod, var_noise=var_noise,a1=1,a2=-1)
data_valid$time2 = (data_valid$time)^2
###

In [3]:
# Tuning parameters on the validation set: alpha,maxdepth,Fuzzy
fixed_regress = c("time","time2")
fixed_split = c("treatment")
cluster = "patient"
var_select = paste("V",1:p,sep="")
alpha = 0.6
maxdepth_factor_select = 0.8
maxdepth_factor_screen = 0.04
system.time({
    mytree = Longtree(data,fixed_regress=fixed_regress,fixed_split=fixed_split,
                      var_select=var_select,cluster=cluster,Fuzzy=FALSE,alpha=alpha,
                      maxdepth_factor_select =  maxdepth_factor_select,
                      maxdepth_factor_screen = maxdepth_factor_screen)
})
mean((predict(mytree,newdata=data_valid,re.form=NA)-data_valid$y)**2)
coef(mytree)

Use Longtree_time
After screening within non-grey modules V1 V2 V3 V289 
The chosen non-grey features are V1 V2 V3 
The chosen grey features are V301 V302 
final features V1 V2 V3 V301 V302

   user  system elapsed 
   7.81    0.23    8.21 

[1] 35.45609

,(Intercept),time,time2,V1,V2,V3,V301,V302
3,4.262397,-6.419367,1.1140322,3.822676,-3.455860,-2.639491,4.660960,1.9657049
4,-14.453392,6.217615,-1.0517839,4.814965,-5.725771,-1.396711,4.728828,0.7768329
6,9.588905,-6.791619,1.0014639,5.785032,6.551489,5.180839,4.859067,0.9588469
7,-6.476356,3.357543,-0.5027088,4.080269,5.262187,6.794826,5.552510,3.0766797


In [4]:
# performance of test set
mse = mean((predict(mytree,newdata=data_test,re.form=NA)-data_test$y)**2)
mse

[1] 40.26155

In [6]:
save_item = list(tree=mytree,mse=mse,alpha=alpha,
                 maxdepth_factor_select=maxdepth_factor_select,
                 maxdepth_factor_screen=maxdepth_factor_screen,
                 final_selection = mytree$final_selection)
saveRDS(save_item, "Longtree_n100.rds")

In [3]:
# retrieve information from the saved data
# tmp = readRDS("Longtree_n100a1.rds")
# tmp$mse

# Theorectically the best

In [10]:
# the final selection among all the chosen features 
var = paste("V",c(1,2,3,301,302,303),sep="")
regress_var = c("time","time2",var)
split_var = c("treatment",var)
Formula = as.formula(paste("y~",paste(regress_var,collapse = "+"),
                   "|",cluster,"|",
                 paste(split_var,collapse = "+")))
system.time({mytree = lmertree(Formula, data = data,alpha = 0.2,maxdepth=6)}) 
mean((predict(mytree,newdata=data_test,re.form=NA)-data_test$y)**2)
# coef(mytree)

   user  system elapsed 
   0.95    0.02    0.98 

[1] 28.9113